<a href="https://colab.research.google.com/github/Simu96/2020-2-OSSP1-WhatsUp-5/blob/sim/Data_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Crawling

In [21]:
# 환경 구축
!pip install selenium
!pip install BeautifulSoup4
!apt update
!apt install chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (130 kB/s)
Reading package lists... Done
Building dependency tree       
Readin

In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException

from bs4 import BeautifulSoup
import time
import csv
import datetime

In [23]:
#공공보건 포털사이트의 뉴스 기사를 크롤링함
site = 'http://health.chosun.com/list.html?menu=&more_menu=&more_smenu=&nowcode=1&type=&pn=2757'

#csv 파일 저장을 위한 f, utf-8을 인코딩으로 사용
f = open('health_chosun.csv', 'w',encoding='utf-8')

#파일 객체를 생성
wr = csv.writer(f)

In [36]:
row = 1

#chrome으로 driver를 실행
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

#조건 성립 시간을 위한 wait 객체 설정
wait = WebDriverWait(driver,20)

#driver의 브라우저 크기를 최대화
driver.maximize_window()
#최대화를 하지 않았을 경우 마우스 포인터가 정확히 div를 클릭하지 못하여 크롤링을 못할 수 있음

#웹의 컴포넌트 로딩을 기다리기 위한 implicitly_wait 함수, 로딩이 끝나면 바로 진행된다.
driver.implicitly_wait(3)

#미리 설정해둔 공공보건 포털사이트로 이동한다.
driver.get(site)

#크롤링할 기사의 날짜 범위 설정
print("크롤링할 기사의 날짜 범위를 설정하세요")

date_entry = input('시작 날짜(YYYY-MM-DD 형식) : ')
year, month, day = map(int, date_entry.split('-'))
date1 = datetime.date(year, month, day)

date_entry = input('종료 날짜(YYYY-MM-DD 형식) : ')
year, month, day = map(int, date_entry.split('-'))
date2 = datetime.date(year, month, day)

#기사의 제목, 내용, 생성날짜를 저장할 변수를 설정한다.
name=""
content=""
date=""

while True:
    time.sleep(1)
    list_num = len(driver.find_elements_by_css_selector("dl.list_item"))
    while True:
        #변수들의 초기화
        name=""
        content=""
        date=""
        while True:
            news_list = driver.find_element_by_css_selector("div#list_area dl:nth-child(" + str(list_num) +")")
            news = news_list.find_element_by_css_selector('dt a')
            news_text = news_list.text
            #잡다한 만화, 책광고 등등의 기사를 제외시킴
            if news.text.find("카툰") != -1 or news.text.find("해랑")!= -1 or news.text.find("서적") != -1 or news.text.find("튼튼선생") != -1 or news.text.find("양냥") != -1 or news_text.find("비만클리닉") != -1:
                print(str(row) + "오류발생")
                list_num -= 1
                if(list_num < 1):
                    break;
            else:
                break;      
        if(list_num < 1):
            break;
        news.click()
        try:
            alert = driver.switch_to.alert
            alert.accept()
            time.sleep(2)
        except :
            pass
        try :
            time.sleep(1)
            name = driver.find_element_by_css_selector("h1#news_title_text_id").text
            date = driver.find_element_by_css_selector("p#date_text").text.split(" ")[1]
            parts = driver.find_elements_by_css_selector("div.par p")

            year, month, day = map(int, date.split('.'))
            date3 = datetime.date(year, month, day)

            for part in parts:
                if part.text.find("기자") == -1 and part.text.replace(" ","") != "":
                    content += part.text
                else:
                    break;
            
            # 해당 기사의 날짜가 사용자 지정 범위 내인 경우에만 csv파일에 작성
            if date1 <= date3 and date3 <= date2:
              wr.writerow([row, name, date, content])
              print(str(row) + " " + name + " / " + str(date3))

            row +=1
            driver.back()
            driver.implicitly_wait(5)
            list_num -= 1
            if(list_num < 1):
              break;            
        except (AttributeError, NoSuchElementException, UnexpectedAlertPresentException):
            driver.back()
            list_num -= 1
            if(list_num < 1):
                break;
            
    page_now = driver.find_element_by_css_selector("div.paginate ul.paginate_num span.listAct").text.split(" ")[0]
    url = "http://health.chosun.com/list.html?menu=&more_menu=&more_smenu=&nowcode=1&type=&pn=" + str(int(page_now) -1)
    driver.get(url)

크롤링할 기사의 날짜 범위를 설정하세요
시작 날짜(YYYY-MM-DD 형식) : 2003-02-27
종료 날짜(YYYY-MM-DD 형식) : 2003-03-13
17 ‘성대 마비증’ 수술로 정상적 목소리 되찾아 / 2003-03-04
18 햇볕 많이 쬐면 피부노화 / 2003-03-04
19 업그레이드 암치료/ 전립선암 / 2003-03-04
20 업그레이드 암치료/ 소아 백혈병 / 2003-03-11
21 울퉁불퉁 불거진 다리 핏줄 수술로 제거해야 / 2003-03-11
22 편집실/ 어린이 당뇨환자 주위 보살핌 중요 / 2003-03-11
23 어린이 당뇨 환자 급속 확산 / 2003-03-11


KeyboardInterrupt: ignored